For fast execution of other computationally expensive methods, the dimensions in this notebook were intentionally reduced for illustrative purposes. The results using large-scale simulation studies are available in our paper.

In [44]:
source("src/net_functions.R")

# Simulate data

In [45]:
re_simu <- simu_data(iter = 1, N = 5, nSample = 5000,  sparse_rate = 0.2)
re_simu$graph

head(re_simu$data) ## patients x diagnosis codes 

0,0.0000000,0.0000000,0.0000000,0.0000000
0,0.0000000,0.0000000,0.0000000,-0.2911937
0,0.0000000,0.0000000,-0.9820707,0.0000000
0,0.0000000,-0.9820707,0.0000000,0.0000000
0,-0.2911937,0.0000000,0.0000000,0.0000000


V1,V2,V3,V4,V5
1,1,0,1,0
1,1,0,0,0
1,1,1,0,1
0,0,1,0,1
1,1,1,0,1
1,1,0,0,0


# Infer networks

In [46]:
## Pairwise OR method
graph_or <- convention_methods(re_simu$data)[[1]]
graph_or

0.0000000,0.0000000,0.000000,-0.1722169,0.0000000
0.0000000,0.0000000,0.000000,0.0000000,-0.4248704
0.0000000,0.0000000,0.000000,-1.0401325,0.0000000
-0.1722169,0.0000000,-1.040132,0.0000000,0.0000000
0.0000000,-0.4248704,0.000000,0.0000000,0.0000000


In [47]:
## Pairwise mutual information method, may require extended computation time
graph_mi <- mi_conn(re_simu$data)
graph_mi

0.000000000,0.0000000000,0.0000000000,0.000924879,0.000000000
0.000000000,0.0000000000,0.0006382483,0.000000000,0.005602639
0.000000000,0.0006382483,0.0000000000,0.032583226,0.000000000
0.000924879,0.0000000000,0.0325832259,0.000000000,0.000000000
0.000000000,0.0056026389,0.0000000000,0.000000000,0.000000000


In [48]:
## Our elastic net regularized graphical modeling method 
graph_en <- Elastic_fit(re_simu$data, plot = FALSE, progressbar = FALSE, alpha = 0.9)$AB_MIN_graph_final
graph_en

0,0.0000000,0.0000000,0.0000000,0.0000000
0,0.0000000,-0.0860425,0.0000000,-0.3068361
0,-0.0860425,0.0000000,-0.9683625,0.0000000
0,0.0000000,-0.9683625,0.0000000,0.0000000
0,-0.3068361,0.0000000,0.0000000,0.0000000


# Assess performance

In [49]:
list(dist_or = base::norm(re_simu$graph - graph_or, "F"), dist_mi = base::norm(re_simu$graph - 
    graph_mi, "F"), dist_en = base::norm(re_simu$graph - graph_en, "F"))

$dist_or
[1] 0.3190589

$dist_mi
[1] 1.495066

$dist_en
[1] 0.1251871

# A small-scale simulation comparison

For full-scale simualtions, please increase the dimensions *N* and *n_simu*. Please consider employing high performance computing clusters if you want to include the mutual information method.

In [50]:
run_simu <- function(iter) {
    re_simu <- simu_data(iter, N = 5, nSample = 5000, sparse_rate = 0.2)  ## increase N for full-scale simulations
    graph_or <- convention_methods(re_simu$data)[[1]]
    graph_mi <- mi_conn(re_simu$data)
    graph_en <- Elastic_fit(re_simu$data, plot = FALSE, progressbar = FALSE, alpha = 0.9)$AB_MIN_graph_final
    return(list(dist_or = base::norm(re_simu$graph - graph_or, "F"), dist_mi = base::norm(re_simu$graph - 
        graph_mi, "F"), dist_en = base::norm(re_simu$graph - graph_en, "F")))
}

In [51]:
library(parallel)
n_simu <- 5 ## increase n_simu to 50 or 100 for full-scale simulations
re_dist <- mclapply(1:n_simu, run_simu, mc.cores = 5)

In [52]:
dist_df <- as.data.frame(do.call(rbind, re_dist))
apply(dist_df, 2, function(v) mean(unlist(v))) ## average distances to the truth, the smaller the better

dist_or   dist_mi   dist_en 
0.6496296 1.9938606 0.5145805